In [ ]:
date_obs = 20240205

In [ ]:
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd

import warnings

import awkward as ak
import awkward_pandas as akpd

from astropy.time import Time
from IPython.display import display, HTML, Markdown
import IPython

from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.tmaUtils import TMAEventMaker
from lsst_efd_client import EfdClient

In [ ]:
warnings.filterwarnings("ignore")
#Based on the notebook written by Chris Walter
# https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/notebooks/SITCOM-1209_blocks_in_time_range.ipynb

# For the TMA events
client = EfdClient("usdf_efd")
event_maker = TMAEventMaker(client=client)

In [ ]:
# Produce a Pandas data frame with block information.  More information can be
# added as needed by modifying the function and following the pattern
# in the code.

def day_obs_dataframe(day_obs, client):
    '''
    Loop over the blocks and sequences for one day and produce a pandas dataframe.
    containing the BLOCK number, BLOCK ID, time start, time stop, and reason for stopping.

    This function returns a pandas dataframe
    '''

    entry_list = []

    block_parser = BlockParser(day_obs, client)
    blocks = block_parser.getBlockNums()

    for block_id in blocks:
        sequences =  block_parser.getSeqNums(block_id)

        for seq_id in sequences:
            info = block_parser.getBlockInfo(block_id, seq_id)

            start_time = info.begin
            end_time = info.end
            reason = info.states[-1]

            entry_list.append([block_id, seq_id, start_time.iso, end_time.iso,
                               reason])

    data_frame = pd.DataFrame(entry_list, columns = ['Block', 'Sequence',
                                                     'Start', 'Stop',
                                                     'Completion Status'])
    return data_frame

def day_obs_report(day_obs, client):
    '''
    Loop over the blocks and sequences for one day and produce a report.
    Interspace TMA events with the block info.
    '''

    block_parser = BlockParser(day_obs, client)
    tma_events = event_maker.getEvents(day_obs)
    blocks = block_parser.getBlockNums()


    from collections import defaultdict

    data_frames = []
    
    for block_id in blocks:
        dict_table = defaultdict(list)
        
        sequences =  block_parser.getSeqNums(block_id)


        for seq_id in sequences:
            info = block_parser.getBlockInfo(block_id, seq_id)
            state_string = ' '.join([str(state) for state in info.states])
            dict_table["Block"].append(block_id)
            dict_table["Seq"].append(seq_id)
            dict_table["States"].append(state_string)
            

            # Also print any TMA events for this block/sequence
            event = block_parser.getEventsForBlock(tma_events, block_id, seq_id)
            dict_table["Events"].append(str(event))
        df = pd.DataFrame(dict_table)
        data_frames.append(df)
    return data_frames

In [ ]:
# This HTML magick below will make it so the example report below doesn't line wrap. Each line can be quite long.

In [ ]:
%%html
<style>
div.jp-OutputArea-output pre {
    white-space: pre;
}
</style>

In [ ]:
day_obs_df = day_obs_dataframe(int(date_obs), client)
dfs = day_obs_report(int(date_obs), client)


# Show example data frame
with pd.option_context("display.max_rows", 1000):
    pd.set_option('display.max_colwidth', None)
    day_obs_df = day_obs_df.style.set_table_styles(
            [{'selector': 'th', 'props': [('text-align', 'left')]}]  # Left align column headers
            ).set_properties(**{'text-align': 'left'})  # Left align data cells

    display(Markdown(f'# SUMMARY REPORT FOR DAYOBS: {date_obs}'))

    _ = display(HTML("<div style='width: 50000px; overflow: auto;'>" +
             day_obs_df.to_html() +
             "</div>"))
    
    for df in dfs:
        block_id = df["Block"].iloc[0]

        df = df.style.set_table_styles(
            [{'selector': 'th', 'props': [('text-align', 'left')]}]  # Left align column headers
            ).set_properties(**{'text-align': 'left'})  # Left align data cells

        display(Markdown(f'# EXECUTED SEQUENCES FOR BLOCK {block_id}'))

        _ = display(HTML("<div style='width: 50000px; overflow: auto;'>" +
             df.to_html() +
             "</div>"))
   

